In [1]:
import os 
import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import requests

import pandas as pd
pd.set_option('display.max_rows', 500)
import numpy as np
from lxml import etree
from Bio import Entrez

import json
from IPython.display import display, HTML

# set a dummy email 
Entrez.email = "jreyna@lji.org"

outdir = '../../results/geo_celltypes/'

In this notebook we are leveraging the Entrez library to query the **BioSamples** database which contains 
lots of metadata for the samples we are interested in. We start from the output of `../query_geo_v2.ipynb` which 

## Query GEO BioSample using the GSM IDs

In [2]:
# open file and read the content in a list
gsm_ids = []
with open('../../results/geo_celltypes/gsm_list.human.tracker.txt', 'r') as fp:
    gsm_ids += sorted(set([x.strip() for x in fp.readlines()]))

with open('../../results/geo_celltypes/gsm_list.mouse.tracker.txt', 'r') as fp:
    gsm_ids += sorted(set([x.strip() for x in fp.readlines()]))


In [3]:
# extract information for each GEO sample from the NCBI BioSample database
# setting up search terms to find a GSM ID witin ALL fields
biosample_filters = '[All Fields] OR '.join(gsm_ids)

# perform the query; currently there are less than 10,000 samples we have to look for,
# in the future if there are ever more than 10,000 samples, the retstart can be updated
# to retstart=10001 and a loop can be made to handle even more samples 
biosample_query = Entrez.esearch(db="biosample", retmax=10000, term=biosample_filters, idtype='acc')
biosample_result = Entrez.read(biosample_query)
biosample_id_list = sorted(set(biosample_result["IdList"]))

In [4]:

# Define the base URL for the efetch command
base_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi'

biosample_tree = None
intervals = 100
for start in range(0, len(biosample_id_list), intervals):

    end = start + intervals
    print(start, end)

    # define the parameters for the efetch command
    params = {
        'db': 'biosample',
        'id': ','.join(biosample_id_list[start:end]),
        'retmode': 'xml',
        'rettype': 'DocumentSummarySet'
    }

    # send the request to the E-utilities API and get the response
    response = requests.get(base_url, params=params)

    if biosample_tree == None:
        biosample_tree = etree.fromstring(response.content)
    else:
        new_tree = etree.fromstring(response.content)

        # join the second XML tree with the main one
        biosample_tree.extend(new_tree)


0 100
100 200
200 300
300 400
400 500
500 600
600 700


In [5]:
# load a dictionary of terms used to  classify organ, disease, and cell type
# as best as possible
with open('metadata_dictionary.json', 'r') as fp:
    celltype_dict = json.load(fp)

# initialize lists to store the metadata
celltype_metadata = []

# creating an example dictionary to store the metadata for each GSM entry
empty_biosample_data = {'name': np.nan, # got it 
                  'organism': np.nan, # got it 
                  'biomaterial': np.nan, # not the best but ok, got it
                  'celltype': np.nan, # not the best but ok, got it
                  'extdb_name': np.nan, # use this to store the GEO Name
                  'extdb_uuid': np.nan, # use this to store the GEO ID
                  'gsm_id': np.nan, # use this to store the GEO ID explicitly for mapping purposes
                  'sra_id': np.nan,
                  'biosample_id': np.nan,
                  'disease': np.nan, # difficult, don't count on finding it 
                  'organ': np.nan, # difficult, don't count on finding it 
                  'treatment': np.nan, # difficult, don't count on finding it
                  'tissue': np.nan, # extremely difficult, don't count on finding it AT ALL
                  'sex': np.nan, # not found, can I find somewhere else?, don't count on finding it 
                  'age': np.nan, # not found, can I find somewhere else?, don't count on finding it 
                  'race': np.nan, # found from time to time
                  'strain': np.nan} # only for mouse} 

# helper function to capitalize words
def capitalize_words(s):
    return(' '.join([w.capitalize() for w in s.split()]))

# helper function to extract an XML element or return an empty string
def get_xml_element_or_default(start_element, query, text=True):
    element = start_element.find(query)
    if element is not None:
        if text:
            return(element.text)
        else:
            return(element)
    return(np.nan)

# iterate through the list of biosample entries
for i, doc_sum in enumerate(biosample_tree.findall(".//DocumentSummary")):

    # initialize the dictionary to store the metadata
    biosample_data = empty_biosample_data.copy()

    ############ Extractable Values ############
    # transform the XML text into an XML tree, for some reason the "<" and ">"
    # are sometimes not parsed correctly, so we needed to reread the data
    # into a tree

    # extract the SampleData XML element
    sample_data = doc_sum.find('.//SampleData')

    # make sample data tree
    sample_tree = etree.fromstring(bytes(sample_data.text.encode('utf-8')))

    # # testing and printing xml values per sample
    # for elem in sample_tree:
    #     for subelem in elem:
    #         if subelem.tag == 'Attribute':
    #             print(subelem.tag, subelem.attrib, subelem.text)
    #     print()
    # if i > 1000:
    #     break
    # continue

    # extract the Ids for this sample
    ids = sample_tree.find("Ids")
    biosample_data['biosample_id'] = ids.find("Id[@db='BioSample']").text
    biosample_data['geo_id'] = ids.find("Id[@db='GEO']").text
    biosample_data['sra_id'] = ids.find("Id[@db='SRA']").text

    # extract the attributes for this sample
    attributes = sample_tree.find("Attributes")

    # obtain the biomaterial
    if attributes.find('Attribute[@harmonized_name="cell_line"]') is not None:
        biomaterial = 'Cell Line'
    else:
        biomaterial = 'Other'
    biosample_data['biomaterial'] = biomaterial

    # obtain the cell type 
    celltype = get_xml_element_or_default(attributes, 'Attribute[@harmonized_name="cell_type"]', text=True)
    biosample_data['celltype'] = celltype

    # obtain the treatment (CHECK OUT: not stored yet)
    treatment = get_xml_element_or_default(attributes, 'Attribute[@harmonized_name="treatment"]', text=True)
    biosample_data['treatment'] = treatment

    # obtain the tissue (CHECK OUT: not stored yet)
    tissue = get_xml_element_or_default(attributes, 'Attribute[@harmonized_name="tissue"]', text=True)
    biosample_data['tissue'] = tissue

    # obtain the sample source (CHECK OUT: not stored yet)
    sample_source = get_xml_element_or_default(attributes, 'Attribute[@harmonized_name="source_name"]', text=True)
    
    # obtain the name/title
    title = sample_tree.find("Description/Title").text
    biosample_data['name'] = title

    # obtain the organism
    organism = sample_tree.find("Description/Organism").get('taxonomy_name')
    biosample_data['organism'] = organism

    strain = get_xml_element_or_default(attributes, 'Attribute[@harmonized_name="strain"]', text=True)
    biosample_data['strain'] = strain

    #organ = sample_tree.find('Attribute [@harmonized_name="cell_type"]').text
    sex = get_xml_element_or_default(attributes, 'Attribute[@harmonized_name="sex"]', text=True)
    biosample_data['sex'] = sex

    ############ Dictionary Based Classification ############
    # attempt to automatically classify 'disease' and 'organ' columns by doing a simple 
    # search of keywords within the whole SampleData XML text
    data_str = sample_data.text.lower()

    # extract the title, attributes and description lowercase strings
    title_str = doc_sum.find("Title").text.lower()
    attributes_str = etree.tostring(attributes).decode().lower()
    description = sample_tree.find("Description")
    description_str = etree.tostring(description).decode().lower()
    check_str = ' '.join([title_str, attributes_str, description_str])

    for celltype_col in ['disease', 'organ']:

        # identify candidate classes for each meta column
        candidate_classes = []
        for curr_class in celltype_dict[celltype_col]:
            synonyms = celltype_dict[celltype_col][curr_class]
            found = any(syn in check_str for syn in synonyms)
            if found:
                candidate_classes.append(curr_class)
        
        # if there are no candidate classes, then assign as N/A
        if len(candidate_classes) == 0:
            biosample_data[celltype_col] = np.nan
        # if there is only one candidate class, then use it
        elif len(candidate_classes) == 1:
            biosample_data[celltype_col] = capitalize_words(candidate_classes[0])
        else:
            # if there are multiple candidate classes, then assign Review
            multiple_classes = capitalize_words(', '.join(candidate_classes))
            # biosample_data[celltype_col] = 'Review: {}'.format(multiple_classes)
            biosample_data[celltype_col] = multiple_classes

    # append the GSM data to the table    
    celltype_metadata.append(biosample_data)
    
    # indicate how many GSM are processed and save the table as the loop runs
    if (i % 100 == 0):
        print("Finished Biosample number:", i)

    # if biosample_data['biosample_id'] == 'SAMN15545066':
    #     break

Finished Biosample number: 0
Finished Biosample number: 100
Finished Biosample number: 200
Finished Biosample number: 300
Finished Biosample number: 400
Finished Biosample number: 500
Finished Biosample number: 600


In [6]:
# Initialize an empty dataframe with the desired column names
cols = ['name', 'organism', 'biomaterial', 'disease',
        'organ', 'tissue', 'celltype', 'strain',
        'sex', 'age', 'race', 'extdb_name', 'extdb_uuid', 'geo_id', 'sra_id', 'biosample_id']
cell_type_df = pd.DataFrame(celltype_metadata, columns=cols)

#### Set default values for NaN values

In [7]:
cell_type_df.loc[cell_type_df.disease.isna(), 'disease'] = 'N/A'
cell_type_df.loc[cell_type_df.organ.isna(), 'organ'] = 'Undetermined'
cell_type_df.loc[cell_type_df.tissue.isna(), 'tissue'] = 'Undetermined'
cell_type_df.loc[cell_type_df.celltype.isna(), 'celltype'] = 'Undetermined'
cell_type_df.loc[(cell_type_df.strain.isna()) & (cell_type_df.organism == "Homo sapiens"), 'strain'] = 'N/A'
cell_type_df.loc[(cell_type_df.strain.isna()) & (cell_type_df.organism != "Homo sapiens"), 'strain'] = 'Undetermined'
cell_type_df.loc[cell_type_df.sex.isna(), 'sex'] = 'Undetermined'
cell_type_df.loc[cell_type_df.age.isna(), 'age'] = 'Undetermined'
cell_type_df.loc[cell_type_df.age.isna(), 'race'] = 'Undetermined'

#### Change Mus to Mus musculus

In [8]:
cell_type_df.loc[:, 'organism'] = cell_type_df.loc[:, 'organism'].str.replace('^Mus$', 'Mus musculus')

In [9]:
cell_type_df.head()

,name,organism,biomaterial,disease,organ,tissue,celltype,strain,sex,age,race,extdb_name,extdb_uuid,geo_id,sra_id,biosample_id
0,Cohesin HiChIP p63KO Day 7 - Replicate 3,Homo sapiens,Cell Line,N/A,Undetermined,Undetermined,Surface Ectoderm Cells,N/A,Undetermined,Undetermined,NaN,NaN,NaN,GSM3397791,SRS3810279,SAMN10102288
1,Cohesin HiChIP p63KO Day 7 - Replicate 2,Homo sapiens,Cell Line,N/A,Undetermined,Undetermined,Surface Ectoderm Cells,N/A,Undetermined,Undetermined,NaN,NaN,NaN,GSM3397790,SRS3810278,SAMN10102289
2,Cohesin HiChIP p63KO Day 7 - Replicate 1,Homo sapiens,Cell Line,N/A,Undetermined,Undetermined,Surface Ectoderm Cells,N/A,Undetermined,Undetermined,NaN,NaN,NaN,GSM3397789,SRS3810277,SAMN10102290
3,Cohesin HiChIP WT Day 7 - Replicate 3,Homo sapiens,Cell Line,N/A,Undetermined,Undetermined,Surface Ectoderm Cells,N/A,Undetermined,Undetermined,NaN,NaN,NaN,GSM3397788,SRS3810276,SAMN10102291
4,Cohesin HiChIP WT Day 7 - Replicate 2,Homo sapiens,Cell Line,N/A,Undetermined,Undetermined,Surface Ectoderm Cells,N/A,Undetermined,Undetermined,NaN,NaN,NaN,GSM3397787,SRS3810275,SAMN10102292


#### Review the Parsing Procedure

In [10]:
review_cols = ['organism',
               'biomaterial',
               'disease',
               'organ',
               'tissue',
               'celltype',
               'strain',
               'sex']

In [11]:
for col in review_cols:
    tdf = cell_type_df.value_counts(col)
    display(HTML('<h3>{}</h3>'.format(col)))
    display(tdf.to_frame())

,0
organism,
Homo sapiens,437
Mus musculus,211


,0
biomaterial,
Other,396
Cell Line,252


,0
disease,
N/A,448
Ewing Sarcoma,55
Leukemia,41
Lung Cancer,23
Neuroblastoma,12
Breast Cancer,11
Colorectal Adenocarcinoma,11
Prostate Carcinoma,9
Esophageal Squamous Cancer,8


,0
organ,
Undetermined,198
Embryo,136
Skeletal System,73
Heart,29
Skin,26
Lung,25
Brain,23
Blood,17
Nerve,16


,0
tissue,
Undetermined,509
Ewing sarcoma cell line (A673),27
heart,20
spleen,14
brain,12
WT E14 mESC,11
Thymus,9
duodenal epithelium,8
Ewing sarcoma cell line (TC71),8


,0
celltype,
Undetermined,269
H9 human Embryonic Stem Cell Line,31
Cultured THP-1 cells,18
Small cell lung cancer cell line,15
mESC,14
ES cell,14
differentiating mES cells,12
Treg,11
"Reprogrammable mouse embryonic fibroblasts (MEFs) prepared from rtTA3-OKSM embryos (embryonic day-E13.5) that were heterozygous for the Oct4-GFP reporter, OKSM cassette and rtTA3.",9


,0
strain,
N/A,437
Undetermined,147
C57BL/6J,23
C57BL/6,16
BALB/cJ,8
E14Tg2A,8
NOD/ShiLtJ,4
M370I-Tg-B6,2
WT-B6,2


,0
sex,
Undetermined,567
male,58
female,23


## Checker

In [12]:
# Define the base URL for the efetch command
base_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi'

# define the parameters for the efetch command
params = {
    'db': 'biosample',
    'id': '17391129', 
    'retmode': 'xml',
    'rettype': 'DocumentSummarySet'
}

# send the request to the E-utilities API and get the response
response = requests.get(base_url, params=params)
check_tree = etree.fromstring(response.content)


In [13]:
sample_data = check_tree.find('.//DocumentSummary/SampleData')

In [14]:
sample_tree = etree.fromstring(bytes(sample_data.text.encode('utf-8')))

In [15]:
etree.dump(sample_tree, pretty_print=True)

<BioSample access="public" publication_date="2021-05-20T00:00:00.000" last_update="2021-05-20T02:19:18.203" submission_date="2021-01-20T15:16:07.327" id="17391129" accession="SAMN17391129">   <Ids>     <Id db="BioSample" is_primary="1">SAMN17391129</Id>     <Id db="SRA">SRS8083433</Id>     <Id db="GEO">GSM5028232</Id>   </Ids>   <Description>     <Title>CD34 HiChIP</Title>     <Organism taxonomy_id="9606" taxonomy_name="Homo sapiens">       <OrganismName>Homo sapiens</OrganismName>     </Organism>   </Description>   <Owner>     <Name>Mullighan, Pathology, St Jude Children's Research Institute</Name>     <Contacts>       <Contact email="chunxu.qu@stjude.org">         <Name>           <First>Chunxu</First>           <Last>Qu</Last>         </Name>       </Contact>     </Contacts>   </Owner>   <Models>     <Model>Generic</Model>   </Models>   <Package display_name="Generic">Generic.1.0</Package>   <Attributes>     <Attribute attribute_name="source_name" harmonized_name="source_name" displ

### Save

In [19]:
# setting the output filename
today = datetime.date.today()
date_str = today.strftime("%Y_%m_%d")
now = datetime.datetime.now()

# set output name (old version is deprecated)
# time_str = now.strftime("%H_%M")
# output = os.path.join(outdir, 'geo.query.cell_type.{}_{}.tsv'.format(date_str, time_str))
output = os.path.join(outdir, 'geo.query.cell_type.{}.tsv'.format(date_str))

print("output file: ", output)
cell_type_df.to_csv(output, index=False, sep='\t')

output file:  ../../results/geo_celltypes/geo.query.cell_type.2023_02_27.tsv
